<h1>CNN - working with mnist data set</h1>

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
mnist = input_data.read_data_sets('/tmp/data',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [19]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

n_classes = 10
batch_size = 100

In [20]:
# height * width
x = tf.placeholder('float',[None,784])
y = tf.placeholder('float')
keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

In [21]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding="SAME")

def maxpool2d(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1], padding="SAME")

In [22]:
def convolutional_neural_network(x):
    # input_data * weights + biases
    weights = {'w_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
                'w_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
              'w_fc':tf.Variable(tf.random_normal([7*7*64,1024])),
              'out':tf.Variable(tf.random_normal([1024,n_classes]))}
    
    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
                'b_conv2':tf.Variable(tf.random_normal([64])),
              'b_fc':tf.Variable(tf.random_normal([1024])),
              'out':tf.Variable(tf.random_normal([n_classes]))}
    x = tf.reshape(x, shape=[-1,28,28,1])
    
    conv1 = tf.nn.relu(conv2d(x,weights['w_conv1']) + biases['b_conv1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = tf.nn.relu(conv2d(conv1,weights['w_conv2']) + biases['b_conv2'])
    conv2 = maxpool2d(conv2)
    
    fc = tf.reshape(conv2, [-1,7*7*64])
    fc =tf.nn.relu(tf.matmul(fc, weights['w_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)
    
    output = tf.matmul(fc, weights['out']) + biases['out']
    
    return output

In [23]:
def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
    
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    # cycles feedback + backprops
    hm_epochs = 10
    
    with tf.Session() as alpha:
        alpha.run(tf.global_variables_initializer())
        
        # training the network
        for e in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                _, c = alpha.run([optimizer,cost],feed_dict={x:epoch_x, y:epoch_y})
                epoch_loss +=c
            print('Epoch {} completed out of {} loss: {}'.format(e,hm_epochs,epoch_loss))
        
        
        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print('Accuracy: ',accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

In [24]:
train_neural_network(x)

Epoch 0 completed out of 10 loss: 3567200.969848633
Epoch 1 completed out of 10 loss: 744176.4150972366
Epoch 2 completed out of 10 loss: 413026.6154384613
Epoch 3 completed out of 10 loss: 256896.45809406042
Epoch 4 completed out of 10 loss: 180969.12414827198
Epoch 5 completed out of 10 loss: 125241.26479196548
Epoch 6 completed out of 10 loss: 94002.06888390094
Epoch 7 completed out of 10 loss: 75757.10926866531
Epoch 8 completed out of 10 loss: 54468.03715942567
Epoch 9 completed out of 10 loss: 48352.383277088404
Accuracy:  0.9729
